# 네이버 영화평(알라딘)에 나온 형태소 SQLite DB에 저장
- 영화평에 나온 형태소 추출
- Okt 형태소 분석
- 형태소, 품사, 나온횟수 DB에 저장하기

In [1]:
import numpy as np
import pandas as pd
import sqlite3

In [2]:
df = pd.read_excel('../10_TextMining/data/04.NaverMovie_Aladdin.xlsx')
df.drop(df.columns[0], axis='columns', inplace=True)
df.head()

,point,review,date
0,10,윌 스미스가 하드캐리하여 의도치 않게 원작을 뛰어넘은 실사영화,2019.05.23 11:26
1,10,영화가 너무 아름답다..ost도 잊혀지지 안고 디즈니 실사중에서 최고중에 최고다 쟈...,2019.05.23 09:11
2,10,존잼임... 난 처음 도입부에 아~라~비안 나~잇 이 부분에서부터 감동받아서 울 뻔...,2019.05.23 21:57
3,10,자스민의 speechless 명장면이고 덤으로 윌 스미스의 연기력 또한 매우 굿~~,2019.05.23 12:55
4,10,A whole new world 나올때때 개소름...,2019.05.23 09:06


In [3]:
reviews = df['review']
reviews.shape

(26440,)

In [4]:
conn = sqlite3.connect('./test.db') 
cur = conn.cursor()

In [5]:
cur = conn.cursor()
cur.execute('CREATE TABLE IF NOT EXISTS Movie_reviews \
    (morph TEXT, \
     pos TEXT, \
     count INT, \
     PRIMARY KEY(morph, pos));')

In [6]:
from konlpy.tag import Okt
okt = Okt()

C:\Users\EZEN\Anaconda3\lib\site-packages\jpype\_core.py:210: UserWarning: 
-------------------------------------------------------------------------------
Deprecated: convertStrings was not specified when starting the JVM. The default
behavior in JPype will be False starting in JPype 0.8. The recommended setting
for new code is convertStrings=False.  The legacy value of True was assumed for
this session. If you are a user of an application that reported this warning,
please file a ticket with the developer.
-------------------------------------------------------------------------------

  """)


In [7]:
morphs = okt.pos(str(reviews[0]))
morphs

[('윌', 'Noun'),
 ('스미스', 'Noun'),
 ('가', 'Josa'),
 ('하드캐리', 'Noun'),
 ('하여', 'Verb'),
 ('의도', 'Noun'),
 ('치', 'Noun'),
 ('않게', 'Verb'),
 ('원작', 'Noun'),
 ('을', 'Josa'),
 ('뛰어넘은', 'Verb'),
 ('실사영화', 'Noun')]

In [13]:
# Test용 데이터 입력
cur.execute("INSERT INTO Movie_reviews VALUES('윌', 'Noun', 1);")
conn.commit()

In [9]:
# Test용 데이터 조회
sql = "SELECT count FROM Movie_reviews WHERE morph like ? and pos like ?;"
cur.execute(sql, ('윌', 'Noun'))
count, = cur.fetchone()
print(count)

1


In [20]:
# Test용 데이터 삭제
cur.execute("DELETE FROM Movie_reviews WHERE morph like '윌';")
conn.commit()

In [11]:
def countMorph(morph, pos):    
    count = 0
    try:
        cur = conn.cursor()
        sql = "SELECT count FROM Movie_reviews WHERE morph like ? and pos like ?"
        cur.execute(sql, (morph, pos))
        count, = cur.fetchone()
    except TypeError as te:
        count = 0
    except Exception as e: 
        print('예외가 발생했습니다.', e)
        count = -1
    finally:
        return count

In [14]:
print(countMorph('윌', 'Noun'))
print(countMorph('원작', 'Noun'))
x = ('윌', 'Noun')
countMorph(*x)     # tuple unpacking

1
0


1

In [15]:
def insertMorph(morph, pos, count):
    cur = conn.cursor()
    sql = "INSERT INTO Movie_reviews VALUES(?, ?, ?);"
    cur.execute(sql, (morph, pos, count))
    conn.commit()

In [17]:
insertMorph(*x, 1)

In [18]:
def updateMorph(morph, pos, count):
    cur = conn.cursor()
    sql = "UPDATE Movie_reviews SET count=? WHERE morph like ? and pos like ?;"
    cur.execute(sql, (count, morph, pos))
    conn.commit()

In [19]:
updateMorph(*x, 2)

In [21]:
i = 1
for review in reviews[:1000]:
    if i % 100 == 0:
        print(i, end=' ')
    i += 1
    morphs = okt.pos(str(review))
    for morph in morphs:
        if morph[1] in ["Josa", "Eomi", "Punctuation", "KoreanParticle", "Exclamation"]:
            continue
        count = countMorph(*morph)     # Unpacking
        if count == 0:                 # 신규 등록
            insertMorph(*morph, 1)
        elif count > 0:                # count update
            updateMorph(*morph, count+1)
        else:
            print('DB Error!!!')

100 200 300 400 500 600 700 800 900 1000 

In [22]:
cur = conn.cursor()
sql = "SELECT count(*) FROM Movie_reviews;"
cur.execute(sql)
count, = cur.fetchone()
print(count)

3925


In [23]:
cur = conn.cursor()
sql = "SELECT * FROM Movie_reviews ORDER BY count DESC LIMIT 10;"
cur.execute(sql)
rows = cur.fetchall()
for row in rows:
    print(row)

('영화', 'Noun', 459)
('너무', 'Adverb', 300)
('알라딘', 'Noun', 181)
('진짜', 'Noun', 173)
('자스민', 'Noun', 129)
('노래', 'Noun', 122)
('들', 'Suffix', 120)
('디즈니', 'Noun', 116)
('최고', 'Noun', 110)
('정말', 'Noun', 92)


In [24]:
sql = "SELECT * FROM Movie_reviews;"
cur = conn.cursor()
cur.execute(sql)
rows = cur.fetchall();
columnName = ['형태소', '품사', '등장횟수']
df = pd.DataFrame(columns = columnName)
for row in rows:
    df = df.append(pd.DataFrame([list(row)], columns = columnName), 
                   ignore_index=True)
df.head()

,형태소,품사,등장횟수
0,윌,Noun,26
1,스미스,Noun,30
2,하드캐리,Noun,17
3,하여,Verb,1
4,의도,Noun,2


In [25]:
conn.close()

In [26]:
setMorphs = set(df['형태소'])
len(setMorphs)

3839

In [27]:
# 같은 형태소, 다른 품사
length = len(df)
zeros = np.zeros(length, dtype = np.int32)
df['동일형태소'] = zeros.T

In [28]:
df.head()

,형태소,품사,등장횟수,동일형태소
0,윌,Noun,26,0
1,스미스,Noun,30,0
2,하드캐리,Noun,17,0
3,하여,Verb,1,0
4,의도,Noun,2,0


In [29]:
for i in range(length-1):
    if i % 100 == 0:
        print(i, end=' ')
    if df.iloc[i,3] == 1:
        continue;
    for k in range(i+1, length):
        if df.iloc[i,0] == df.iloc[k,0]:
            df.iloc[i,3] = 1
            df.iloc[k,3] = 1
            continue

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 3400 3500 3600 3700 3800 3900 

In [30]:
ddf = df[df['동일형태소'] == 1]

In [31]:
ddf

,형태소,품사,등장횟수,동일형태소
19,중,Suffix,54,1
28,울,Modifier,6,1
39,이,Noun,39,1
43,울,Noun,7,1
60,개,Noun,19,1
98,함,Noun,19,1
111,동안,Foreign,3,1
119,들,Suffix,120,1
131,적,Suffix,74,1
150,해도,Noun,4,1


In [33]:
ddf[ddf['형태소'] == '느낌']

,형태소,품사,등장횟수,동일형태소
544,느낌,Noun,30,1
2996,느낌,Verb,1,1


In [50]:
df.to_csv('data/03.영화평 형태소.csv', encoding='euc-kr')

### 정렬한 후에 찾는 방법

In [34]:
df = pd.read_csv('data/03.영화평 형태소.csv', encoding='euc-kr')
df.drop(df.columns[0], axis='columns', inplace=True)
df.head()

,형태소,품사,등장횟수,동일형태소
0,윌,Noun,704,0
1,스미스,Noun,810,0
2,하드캐리,Noun,405,0
3,하여,Verb,33,0
4,의도,Noun,18,0


In [35]:
sdf = df.sort_values("형태소")
sdf.head()

,형태소,품사,등장횟수,동일형태소
22573,,Foreign,1,0
9072,#Fall,Hashtag,1,0
9075,#Speechless,Hashtag,1,0
9070,#감정,Hashtag,1,0
20224,#꿀잼,Hashtag,1,0


In [36]:
for i in range(len(sdf)-1):
    if sdf.iloc[i,0] == sdf.iloc[i+1,0]:
        sdf.iloc[i,3] = 1
        sdf.iloc[i+1,3] = 1

In [28]:
ddf = sdf[sdf['동일형태소'] == 1]
ddf

,형태소,품사,등장횟수,동일형태소
227,ㅋ,KoreanParticle,326,1
20140,ㅋ,Foreign,1,1
1625,각,Modifier,9,1
7236,각,Noun,17,1
6473,간,Foreign,3,1
5400,간,Suffix,21,1
4548,간,Noun,30,1
584,감,Noun,165,1
20945,감,Verb,1,1
1407,갓,Noun,51,1


In [37]:
d2df = [item for item in ddf['형태소'] if len(item) >= 2]
d2df

['동안',
 '해도',
 '다른',
 '이런',
 '모든',
 '느낌',
 '모두',
 '아무',
 '이런',
 '동안',
 '다른',
 '어느',
 '짜리',
 '아무',
 '다섯',
 '다섯',
 '모두',
 '모든',
 '짜리',
 '해도',
 '살면서',
 '느낌',
 '어느',
 '살면서']

In [38]:
ddf[ddf['형태소'] == '다섯']

,형태소,품사,등장횟수,동일형태소
1277,다섯,Modifier,2,1
1375,다섯,Determiner,1,1


In [40]:
ddf[ddf['형태소'] == '어느']

,형태소,품사,등장횟수,동일형태소
908,어느,Adverb,2,1
3181,어느,Modifier,1,1
